In [2]:
import os, math, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from keras_hub.models import ViTImageClassifier

from sklearn.metrics import classification_report, confusion_matrix
import itertools

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
DATA_ROOT = "./processed"
RESULTS_DIR = "./results_vit"
os.makedirs(RESULTS_DIR, exist_ok=True)

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 15
VAL_SPLIT = 0.20
TEST_SPLIT_FROM_VAL = 0.5

AUTOTUNE = tf.data.AUTOTUNE

In [4]:
# Load train+val from directory
full_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    label_mode="int",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=VAL_SPLIT,
    subset="both",
    seed=42,
)

train_ds, val_ds_raw = full_ds
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Classes:", class_names)
print("Number of classes:", NUM_CLASSES)

Found 42385 files belonging to 13 classes.
Using 33908 files for training.
Using 8477 files for validation.
Classes: ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Western_Medieval']
Number of classes: 13


In [5]:
# Split val into val + test
val_batches = tf.data.experimental.cardinality(val_ds_raw)
test_ds = val_ds_raw.take(val_batches // 2)
val_ds  = val_ds_raw.skip(val_batches // 2)

print("Train batches:", tf.data.experimental.cardinality(train_ds).numpy())
print("Val batches:", tf.data.experimental.cardinality(val_ds).numpy())
print("Test batches:", tf.data.experimental.cardinality(test_ds).numpy())

Train batches: 1060
Val batches: 133
Test batches: 132


In [12]:
# Data augmentation (used inside the ViT model)
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.05)
    ],
    name="data_augmentation"
)

# # Add rescaling and prefetch
# def prepare_dataset(ds, shuffle=False):
#     ds = ds.map(
#         lambda x, y: (tf.cast(x, tf.float32) / 255.0, y),
#         num_parallel_calls=AUTOTUNE
#     )
#     if shuffle:
#         ds = ds.shuffle(buffer_size=1000)
#     return ds.prefetch(AUTOTUNE)

def prepare_dataset(ds, shuffle=False, augment=False):
    def apply_augment(x, y):
        x = tf.cast(x, tf.float32) / 255.0
        if augment:
            x = data_augmentation(x)
        return x, y
    ds = ds.map(apply_augment, num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
    return ds.prefetch(AUTOTUNE)

train_ds_prep = prepare_dataset(train_ds, shuffle=True, augment=True)
val_ds_prep   = prepare_dataset(val_ds, shuffle=False, augment=False)
test_ds_prep  = prepare_dataset(test_ds, shuffle=False, augment=False)

In [39]:
# # Helper layers
# class Patches(layers.Layer):
#     def __init__(self, patch_size, **kwargs):
#         super().__init__(**kwargs)
#         self.patch_size = patch_size

#     def call(self, images):
#         batch_size = tf.shape(images)[0]
#         patches = tf.image.extract_patches(
#             images=images,
#             sizes=[1, self.patch_size, self.patch_size, 1],
#             strides=[1, self.patch_size, self.patch_size, 1],
#             rates=[1, 1, 1, 1],
#             padding="VALID",
#         )
#         patch_dims = patches.shape[-1]
#         patches = tf.reshape(patches, [batch_size, -1, patch_dims])
#         return patches

#     def get_config(self):
#         config = super().get_config()
#         config.update({"patch_size": self.patch_size})
#         return config


# class PatchEncoder(layers.Layer):
#     def __init__(self, num_patches, projection_dim, **kwargs):
#         super().__init__(**kwargs)
#         self.num_patches = num_patches
#         self.projection = layers.Dense(units=projection_dim)
#         self.position_embedding = layers.Embedding(
#             input_dim=num_patches, output_dim=projection_dim
#         )

#     def call(self, patch):
#         positions = tf.range(start=0, limit=self.num_patches, delta=1)
#         encoded = self.projection(patch) + self.position_embedding(positions)
#         return encoded

#     def get_config(self):
#         config = super().get_config()
#         config.update({
#             "num_patches": self.num_patches,
#             "projection_dim": self.projection.units,
#         })
#         return config


# def mlp(x, hidden_units, dropout_rate):
#     for units in hidden_units:
#         x = layers.Dense(units, activation="gelu")(x)
#         x = layers.Dropout(dropout_rate)(x)
#     return x

In [41]:
# # Build ViT classifier
# def create_vit_classifier(
#     input_shape=IMAGE_SIZE + (3,),
#     num_classes=NUM_CLASSES,
#     patch_size=16,
#     projection_dim=128,
#     transformer_layers=8,
#     num_heads=8,
#     transformer_mlp_units=[256, 128],
#     head_mlp_units=[512, 256],
# ):
#     inputs = layers.Input(shape=input_shape)

#     # Data augmentation
#     augmented = data_augmentation(inputs)

#     # Create patches.
#     patches = Patches(patch_size, name="patches")(augmented)
#     num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)

#     # Encode patches.
#     encoded_patches = PatchEncoder(num_patches, projection_dim, name="patch_encoder")(patches)

#     # Create multiple layers of the Transformer block.
#     for _ in range(transformer_layers):
#         # Layer normalization 1.
#         x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
#         # Multi-head attention layer.
#         attention_output = layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=projection_dim, dropout=0.1
#         )(x1, x1)
#         # Skip connection 1.
#         x2 = layers.Add()([attention_output, encoded_patches])
#         # Layer normalization 2.
#         x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
#         # MLP.
#         x3 = mlp(x3, hidden_units=transformer_mlp_units, dropout_rate=0.1)
#         # Skip connection 2.
#         encoded_patches = layers.Add()([x3, x2])

#     # Create a [batch, projection_dim] tensor.
#     representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
#     representation = layers.GlobalAveragePooling1D()(representation)

#     representation = layers.Dropout(0.3)(representation)
#     features = mlp(representation, hidden_units=head_mlp_units, dropout_rate=0.3)

#     logits = layers.Dense(num_classes, activation="softmax", name="predictions")(features)

#     model = keras.Model(inputs=inputs, outputs=logits, name="vit_classifier")
#     return model


# vit_model = create_vit_classifier()
# vit_model.summary()

Model: "vit_classifier"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ data_augmentation   │ (None, 224, 224,  │          0 │ input_layer_5[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patches (Patches)   │ (None, None, 768) │          0 │ data_augmentatio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_encoder       │ (None, 196, 128)  │    123,520 │ patches[0][0]     │
│ (PatchEncoder)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 196, 128)  │        256 │ patch_encoder[0]… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 196, 128)  │    527,488 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_34 (Add)        │ (None, 196, 128)  │          0 │ multi_head_atten… │
│                     │                   │            │ patch_encoder[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 196, 128)  │        256 │ add_34[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 196, 256)  │     33,024 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_58          │ (None, 196, 256)  │          0 │ dense_42[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 196, 128)  │     32,896 │ dropout_58[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_59          │ (None, 196, 128)  │          0 │ dense_43[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_35 (Add)        │ (None, 196, 128)  │          0 │ dropout_59[0][0], │
│                     │                   │            │ add_34[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 196, 128)  │        256 │ add_35[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 196, 128)  │    527,488 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_36 (Add)        │ (None, 196, 128)  │          0 │ multi_head_atten… │
│                     │                   │            │ add_35[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 196, 128)  │        256 │ add_36[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 196, 256)  │     33,024 │ layer_normalizat

 Total params: 5,075,853 (19.36 MB)

 Trainable params: 5,075,853 (19.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# # Class weights
# class_weight = None

# # Compile
# vit_model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=3e-4),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )

In [14]:
# Callbacks
checkpoint_path = os.path.join(RESULTS_DIR, "vit_best.keras")
callbacks = [
    keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=False,
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=2,
        min_lr=1e-6,
    ),
]

In [43]:
# # Train
# history = vit_model.fit(
#     train_ds_prep,
#     validation_data=val_ds_prep,
#     epochs=EPOCHS,
#     callbacks=callbacks,
#     class_weight=class_weight,
# )

Epoch 1/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1526s 1s/step - accuracy: 0.1858 - loss: 2.3354 - val_accuracy: 0.2276 - val_loss: 2.2187 - learning_rate: 3.0000e-04
Epoch 2/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1527s 1s/step - accuracy: 0.2369 - loss: 2.2057 - val_accuracy: 0.2732 - val_loss: 2.1257 - learning_rate: 3.0000e-04
Epoch 3/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1545s 1s/step - accuracy: 0.2672 - loss: 2.1358 - val_accuracy: 0.2894 - val_loss: 2.0750 - learning_rate: 3.0000e-04
Epoch 4/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1532s 1s/step - accuracy: 0.2906 - loss: 2.0810 - val_accuracy: 0.2988 - val_loss: 2.0294 - learning_rate: 3.0000e-04
Epoch 5/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1534s 1s/step - accuracy: 0.3060 - loss: 2.0382 - val_accuracy: 0.3348 - val_loss: 1.9660 - learning_rate: 3.0000e-04
Epoch 6/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1529s 1s/step - accuracy: 0.3231 - loss: 2.0006 - val_accuracy: 0.3374 - val_loss: 1.9509 - learning_rate: 3.0000e-04
Epoch 7/15
1060/1060 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [16]:
vit_pre_model = ViTImageClassifier.from_preset(
    "vit_base_patch16_224_imagenet",
    num_classes=NUM_CLASSES,
    preprocessor=None,
)

vit_pre_model.backbone.trainable = False

vit_pre_model.compile(
    optimizer=keras.optimizers.Adam(5e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

vit_pre_model.summary()

Model: "vi_t_image_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                  ┃ Output Shape                       ┃             Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ images (InputLayer)                           │ (None, 224, 224, 3)                │                   0 │
├───────────────────────────────────────────────┼────────────────────────────────────┼─────────────────────┤
│ vi_t_backbone (ViTBackbone)                   │ (None, 197, 768)                   │          85,798,656 │
├───────────────────────────────────────────────┼────────────────────────────────────┼─────────────────────┤
│ get_item (GetItem)                            │ (None, 768)                        │                   0 │
├───────────────────────────────────────────────┼────────────────────────────────────┼─────────────────────┤
│ output_dropout (Dropout)                      │ (None, 768)                        │                   0 │
├───────────────────────────────────────────────┼────────────────────────────────────┼─────────────────────┤
│ predictions (Dense)                           │ (None, 13)                         │               9,997 │
└───────────────────────────────────────────────┴────────────────────────────────────┴─────────────────────┘

 Total params: 85,808,653 (327.33 MB)

 Trainable params: 9,997 (39.05 KB)

 Non-trainable params: 85,798,656 (327.30 MB)

In [ ]:
history_vit_pre = vit_pre_model.fit(
    train_ds_prep,
    validation_data=val_ds_prep,
    epochs=EPOCHS,
    callbacks=callbacks,
)

Epoch 1/15


2025-12-07 14:09:54.395455: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:23: Filling up shuffle buffer (this may take a while): 744 of 1000
2025-12-07 14:09:57.862226: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


 232/1060 ━━━━━━━━━━━━━━━━━━━━ 39:48 3s/step - accuracy: 0.1160 - loss: 2.9525

In [ ]:
def plot_history(history, metric="accuracy"):
    plt.figure(figsize=(6,4))
    plt.plot(history.history[metric], label=f"train_{metric}")
    plt.plot(history.history[f"val_{metric}"], label=f"val_{metric}")
    plt.xlabel("Epoch")
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_history(history_vit_pre, metric="accuracy")
plot_history(history_vit_pre, metric="loss")

In [ ]:
# Get predictions
y_true = []
y_pred = []

for batch_images, batch_labels in test_ds_prep:
    preds = vit_pre_model.predict(batch_images, verbose=0)
    y_true.extend(batch_labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Classification report
print("Classification report (ViT):")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 10))
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Confusion matrix – ViT (pretrained)")
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=90)
plt.yticks(tick_marks, class_names)

thresh = cm.max() / 2.0
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(
        j,
        i,
        format(cm[i, j], "d"),
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black",
    )

plt.ylabel("True label")
plt.xlabel("Predicted label")
plt.tight_layout()
plt.show()

In [47]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    img_array: shape (1, H, W, 3) scaled [0,1]
    model: trained keras model
    last_conv_layer_name: string, name of last conv layer
    pred_index: optional class index; if None, uses top predicted class
    """
    # Build a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    last_conv_layer = model.get_layer(last_conv_layer_name)
    grad_model = keras.models.Model(
        [model.inputs], [last_conv_layer.output, model.output]
    )

    # Record operations for automatic differentiation
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, conv_outputs)

    # Pool the gradients over all the axes leaving out the channel dimension
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array by "how important this channel is"
    conv_outputs = conv_outputs[0]  # (H', W', C)
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)  # (H', W')

    # ReLU and normalize between 0 and 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + 1e-8)
    return heatmap.numpy()

In [49]:
def overlay_gradcam_on_image(img, heatmap, alpha=0.4):
    """
    img: (H, W, 3) RGB image, scaled [0,1]
    heatmap: (H', W') float [0,1], will be resized to image size
    """
    import cv2  # if you prefer to avoid cv2, you can use PIL + numpy; keeping cv2 for convenience

    h, w = img.shape[:2]
    heatmap_resized = cv2.resize(heatmap, (w, h))

    heatmap_color = plt.cm.jet(heatmap_resized)[..., :3]  # drop alpha

    superimposed_img = (1 - alpha) * img + alpha * heatmap_color
    superimposed_img = np.clip(superimposed_img, 0, 1)
    return superimposed_img

In [51]:
# Example: load your trained CNN / TL model
# cnn_model = keras.models.load_model("/path/to/your_cnn_or_tl_model.keras")

# Get the last conv layer name from cnn_model.summary()
# For example, EfficientNetB0's last conv layer is often "top_conv"
LAST_CONV_LAYER_NAME = "top_conv"   # <-- CHANGE to your model

def run_gradcam_example(img_path, model, last_conv_layer_name):
    # Load image
    img = keras.utils.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = keras.utils.img_to_array(img)
    img_array = img_array / 255.0
    img_batch = np.expand_dims(img_array, axis=0)

    # Predict class
    preds = model.predict(img_batch)
    pred_class = np.argmax(preds[0])
    pred_proba = preds[0][pred_class]
    pred_label = class_names[pred_class]

    print(f"Predicted: {pred_label} (p={pred_proba:.3f})")

    # Grad-CAM heatmap
    heatmap = make_gradcam_heatmap(
        img_batch,
        model=model,
        last_conv_layer_name=last_conv_layer_name,
        pred_index=pred_class,
    )

    # Overlay
    overlay = overlay_gradcam_on_image(img_array, heatmap, alpha=0.5)

    # Plot original, heatmap, overlay
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.title("Original")
    plt.imshow(img_array)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Grad-CAM heatmap")
    plt.imshow(heatmap, cmap="jet")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Overlay")
    plt.imshow(overlay)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


# Example usage:
# test_image_path = "/path/to/some/test/painting.jpg"
# run_gradcam_example(test_image_path, cnn_model, LAST_CONV_LAYER_NAME)